In [ ]:
from glob import glob
import os
import pandas as pd

In [ ]:
def parse_name(df):
    df[['n', 't', 'd', 'c', 'idx']] = df['name'].str.extract(r'cap100_n(\d+)_t(\d+)_(Lon|Sh)(Lr|Sm)_(\d+)\.txt')
    df['n'] = df['n'].astype(int)
    df['t'] = df['t'].astype(int)
    df['d'] = df['d'].map(lambda x: {'Lon': 'd_L', 'Sh': 'd_S'}[x])
    df['c'] = df['c'].map(lambda x: {'Lr': 'c_H', 'Sm': 'c_L'}[x])
    df['idx'] = df['idx'].astype(int)

In [ ]:
logs = []
for file in glob('../logs/mip/*.log'):
    log = pd.read_csv(file, sep='\t')
    log['model'] = os.path.basename(file)[:-4].split('_', 1)[1]
    logs.append(log)

df = pd.concat(logs)
parse_name(df)
df['dt_solve'] = df['dt_solve'].clip(0.0, 1800.0)

# Table 1 (partially)

In [ ]:
df.pivot_table(
    index=['n', 't', 'd', 'c'],
    columns=['model'],
    values=['mrel.ObjVal'],
).sort_index(ascending=(True, True, False, False)).swaplevel(axis='columns').sort_index(axis='columns')

## Table 7

In [ ]:
df.pivot_table(
    index=['n', 't', 'd', 'c'],
    columns=['model'],
    aggfunc={'dt_solve': 'mean', 'optimal': 'sum'},
).sort_index(ascending=(True, True, False, False)).swaplevel(axis='columns').sort_index(axis='columns')